In [1]:
import pandas as pd

In [2]:
df = pd.read_json('../data/meta_Amazon_Fashion.jsonl', lines=True)

In [3]:
df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,AMAZON FASHION,YUEDGE 5 Pairs Men's Moisture Control Cushione...,4.6,16,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],GiveGift,[],{'Package Dimensions': '10.31 x 8.5 x 1.73 inc...,B08BHN9PK5,NaN
1,AMAZON FASHION,DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,4.1,7,"[Drawstring closure, Machine Wash]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],DouBCQ,[],{'Package Dimensions': '15 x 10.2 x 0.4 inches...,B08R39MRDW,NaN
2,AMAZON FASHION,Pastel by Vivienne Honey Vanilla Girls' Trapez...,4.3,11,"[Zipper closure, Hand Wash Only]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Pastel by Vivienne,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B077KJHCJ4,NaN
3,AMAZON FASHION,Mento Streamtail,2.0,1,"[Thermoplastic Rubber sole, High Density Premi...",[Slip on the Women's Mento and you're ready to...,29.81,[{'thumb': 'https://m.media-amazon.com/images/...,[],Guy Harvey,[],{'Package Dimensions': '11.22 x 4.72 x 4.33 in...,B0811M2JG9,NaN
4,AMAZON FASHION,RONNOX Women's 3-Pairs Bright Colored Calf Com...,4.3,3032,"[Pull On closure, Size Guide: ""S"" fits calf 10...",[Ronnox Calf Sleeves - Allowing Your Body to P...,17.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'HONEST Review: RONNOX Women's 3-Pa...,RONNOX,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B07SB2892S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826103,AMAZON FASHION,Salimdy Women's Fashion New Ruched Waist Class...,2.0,1,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Salimdy,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B078HY8B34,NaN
826104,AMAZON FASHION,Cat Mom Tshirt Trendy Short Sleeve Crewneck Gr...,5.0,1,"[Cotton Blend, Material: Mother's day gift cat...",[Womens cat shirt casual summer short sleeve c...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,[],{'Package Dimensions': '12.32 x 11.06 x 0.79 i...,B09V5DB85S,NaN
826105,AMAZON FASHION,Nike San Francisco 49ers Hot Quarter Zip S/S P...,5.0,1,"[Zipper closure, Hand Wash Only]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Nike,[],"{'Package Dimensions': '12 x 8 x 1 inches', 'I...",B00NTQ14CK,NaN
826106,AMAZON FASHION,Joe's USA Mens 3X-Large Tall Long Sleeve Easy ...,5.0,2,"[100% Cotton, Button closure, Machine Wash]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Joe's USA,[],{'Product Dimensions': '6 x 6 x 0.5 inches; 0....,B00Y8O4N8E,NaN


Trying to understand the data. I want to reduce the dataset as much as possible an focus on high value data.

In [4]:
# Drop columns that are entirely empty
df = df.dropna(axis=1, how='all')
df

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin
0,AMAZON FASHION,YUEDGE 5 Pairs Men's Moisture Control Cushione...,4.6,16,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],GiveGift,[],{'Package Dimensions': '10.31 x 8.5 x 1.73 inc...,B08BHN9PK5
1,AMAZON FASHION,DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,4.1,7,"[Drawstring closure, Machine Wash]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],DouBCQ,[],{'Package Dimensions': '15 x 10.2 x 0.4 inches...,B08R39MRDW
2,AMAZON FASHION,Pastel by Vivienne Honey Vanilla Girls' Trapez...,4.3,11,"[Zipper closure, Hand Wash Only]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Pastel by Vivienne,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B077KJHCJ4
3,AMAZON FASHION,Mento Streamtail,2.0,1,"[Thermoplastic Rubber sole, High Density Premi...",[Slip on the Women's Mento and you're ready to...,29.81,[{'thumb': 'https://m.media-amazon.com/images/...,[],Guy Harvey,[],{'Package Dimensions': '11.22 x 4.72 x 4.33 in...,B0811M2JG9
4,AMAZON FASHION,RONNOX Women's 3-Pairs Bright Colored Calf Com...,4.3,3032,"[Pull On closure, Size Guide: ""S"" fits calf 10...",[Ronnox Calf Sleeves - Allowing Your Body to P...,17.99,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'HONEST Review: RONNOX Women's 3-Pa...,RONNOX,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B07SB2892S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
826103,AMAZON FASHION,Salimdy Women's Fashion New Ruched Waist Class...,2.0,1,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Salimdy,[],"{'Is Discontinued By Manufacturer': 'No', 'Pac...",B078HY8B34
826104,AMAZON FASHION,Cat Mom Tshirt Trendy Short Sleeve Crewneck Gr...,5.0,1,"[Cotton Blend, Material: Mother's day gift cat...",[Womens cat shirt casual summer short sleeve c...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Generic,[],{'Package Dimensions': '12.32 x 11.06 x 0.79 i...,B09V5DB85S
826105,AMAZON FASHION,Nike San Francisco 49ers Hot Quarter Zip S/S P...,5.0,1,"[Zipper closure, Hand Wash Only]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Nike,[],"{'Package Dimensions': '12 x 8 x 1 inches', 'I...",B00NTQ14CK
826106,AMAZON FASHION,Joe's USA Mens 3X-Large Tall Long Sleeve Easy ...,5.0,2,"[100% Cotton, Button closure, Machine Wash]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Joe's USA,[],{'Product Dimensions': '6 x 6 x 0.5 inches; 0....,B00Y8O4N8E


In [5]:
# Get unique values for key columns
print("Unique main categories:")
print(df['main_category'].unique())


Unique main categories:
['AMAZON FASHION']


In [6]:
# Check for non-empty category lists
print("Number of rows with non-empty categories:", 
      len(df[df['categories'].apply(lambda x: len(x) > 0)]))



Number of rows with non-empty categories: 0


In [7]:
# Group by parent_asin and count occurrences, sort in descending order
parent_asin_counts = df['parent_asin'].value_counts()
print("\nParent ASIN counts (descending):")
print(parent_asin_counts)



Parent ASIN counts (descending):
parent_asin
B08BHN9PK5    1
B00TT1OOB2    1
B08JTB3V2P    1
B01HI6GS2M    1
B01IU8B5Y8    1
             ..
B07HR134XV    1
B0823QNLLC    1
B07DSC93F8    1
B0176WL1FC    1
B0895H6NWS    1
Name: count, Length: 826108, dtype: int64


In [8]:
# Drop specified columns
df = df.drop(['main_category', 'videos', 'categories', 'parent_asin'], axis=1)
df.columns

Index(['title', 'average_rating', 'rating_number', 'features', 'description',
       'price', 'images', 'store', 'details'],
      dtype='object')

In [9]:
# Extract first image thumbnail URL from images list
df['thumbnail'] = df['images'].apply(lambda x: x[0]['thumb'] if x and len(x) > 0 and 'thumb' in x[0] else None)

# Display sample of data with new thumbnail column
df.head()


,title,average_rating,rating_number,features,description,price,images,store,details,thumbnail
0,YUEDGE 5 Pairs Men's Moisture Control Cushione...,4.6,16,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,GiveGift,{'Package Dimensions': '10.31 x 8.5 x 1.73 inc...,https://m.media-amazon.com/images/I/41+cCfaVOF...
1,DouBCQ Women's Palazzo Lounge Wide Leg Casual ...,4.1,7,"[Drawstring closure, Machine Wash]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,DouBCQ,{'Package Dimensions': '15 x 10.2 x 0.4 inches...,https://m.media-amazon.com/images/I/515cR-ta1E...
2,Pastel by Vivienne Honey Vanilla Girls' Trapez...,4.3,11,"[Zipper closure, Hand Wash Only]",[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,Pastel by Vivienne,"{'Is Discontinued By Manufacturer': 'No', 'Pac...",https://m.media-amazon.com/images/I/31GwmwNCdA...
3,Mento Streamtail,2.0,1,"[Thermoplastic Rubber sole, High Density Premi...",[Slip on the Women's Mento and you're ready to...,29.81,[{'thumb': 'https://m.media-amazon.com/images/...,Guy Harvey,{'Package Dimensions': '11.22 x 4.72 x 4.33 in...,https://m.media-amazon.com/images/I/31P-uHUUIX...
4,RONNOX Women's 3-Pairs Bright Colored Calf Com...,4.3,3032,"[Pull On closure, Size Guide: ""S"" fits calf 10...",[Ronnox Calf Sleeves - Allowing Your Body to P...,17.99,[{'thumb': 'https://m.media-amazon.com/images/...,RONNOX,"{'Is Discontinued By Manufacturer': 'No', 'Pac...",https://m.media-amazon.com/images/I/51CqMDJOOD...


In [10]:
# Drop additional columns
df = df.drop(['description', 'details', 'images'], axis=1)

In [11]:
# Reduces the dataset
df = df[df['price'].isna() == False]

In [12]:
df

,title,average_rating,rating_number,features,price,store,thumbnail
3,Mento Streamtail,2.0,1,"[Thermoplastic Rubber sole, High Density Premi...",29.81,Guy Harvey,https://m.media-amazon.com/images/I/31P-uHUUIX...
4,RONNOX Women's 3-Pairs Bright Colored Calf Com...,4.3,3032,"[Pull On closure, Size Guide: ""S"" fits calf 10...",17.99,RONNOX,https://m.media-amazon.com/images/I/51CqMDJOOD...
8,LYCKYY Women's Tie Dye Sweatshirt Crewneck Lon...,3.7,52,[Pull On closure],9.99,LYCKYY,https://m.media-amazon.com/images/I/41DW4uwWLQ...
11,Sexyshine Women's Casual Fall Knit Long Sleeve...,3.6,7,"[Cotton Blend, Asian Size,Smaller than US Size...",26.99,Sexyshine,https://m.media-amazon.com/images/I/51Cijpzflv...
33,Result Core Men's Soft Shell II Gilet Red 3XL,5.0,1,"[Polyester,Microfleece,Elastane]",53.95,Result Core,https://m.media-amazon.com/images/I/41vLlRDu6V...
...,...,...,...,...,...,...,...
826034,Tommy Hilfiger Solid Textured Stripe Purple On...,4.3,2,[],39.99,Tommy Hilfiger,https://m.media-amazon.com/images/I/31HXSzlD8X...
826061,Light Stonewash Five Star Regular Fit w/Flex J...,5.0,3,[Zipper closure],34.98,Wrangler,https://m.media-amazon.com/images/I/31gydacTIP...
826076,Kate Spade New York Reiley Laurel Way Purse (H...,5.0,3,[],269.00,Kate Spade New York,https://m.media-amazon.com/images/I/41Jc8ugY5v...
826094,Anne Klein New York Ceramic 3 Bracelet & 32mm ...,3.7,3,[],171.69,Anne Klein,https://m.media-amazon.com/images/I/515IJhiU1B...


In [14]:
df.to_json('../data/processed_data.jsonl', orient='records', lines=True)